In [58]:
import pandas as pd
import os
import pprint


# os.chdir("content/MyDrive/Colab Notebooks")
# print(os.curdir)
# print(os.listdir("./input"))
pprint.pprint(os.listdir("./output/submission"))



['submission_41_tidycol_base_raw.csv',
 'submission_33-lightgbm-stratified-kfold.csv',
 'submission_31-lightgbm-normal_raw.csv',
 'submission_11-with-familysize-logi-reg.csv',
 'submission_42_tidycol_tidy.csv',
 'submission_42_tidycol_tidy_raw.csv',
 'submission_33-lightgbm-stratified-kfold_raw.csv',
 'submission_43_tidycol_with_cabin_prefix.csv',
 'submission_41_tidycol_base.csv',
 'submission_44_tidycol_with_cabin_prefix_with_embarked_raw.csv',
 'submission_44_tidycol_with_cabin_prefix_with_embarked.csv',
 'submission_43_tidycol_with_cabin_prefix_raw.csv',
 'submission_21-random-rorest.csv',
 'submission_31-lightgbm-normal.csv',
 'submission_32-lightgbm-change-hyper-param_raw.csv',
 'submission_32-lightgbm-change-hyper-param.csv']


In [ ]:
# TODO: 1,0ではなくて、小数のデータでensanbleする。

In [56]:
#  タイプ １

# 平均で算出
sub = pd.read_csv('input/gender_submission.csv')
sub['Survived'] = 0

file_names = [
#     'submission_11-with-familysize-logi-reg.csv',
    'submission_21-random-rorest.csv',
#     'submission_31-lightgbm-normal_raw.csv',
#     'submission_32-lightgbm-change-hyper-param_raw.csv',
    'submission_33-lightgbm-stratified-kfold_raw.csv',
#     'submission_41_tidycol_base_raw.csv',
#     'submission_42_tidycol_tidy_raw.csv',
#     'submission_43_tidycol_with_cabin_prefix_raw.csv',
#     'submission_46_tidycol_many_correlation_raw.csv',
#     'submission_47_tidycol_many_correlation_with_sibsp_parch_raw.csv',
#     'submission_48_tidycol_many_correlation_cate_plus_raw.csv',
    
]

file_dfs = []
for file_name in file_names:
    file_dfs.append(pd.read_csv(f'./output/submission/{file_name}'))

for file_df in file_dfs:
  sub['Survived'] = sub['Survived'] + file_df['Survived']

# prefixを作る
prefix_parts = []
for file_name in file_names:
    tmp = file_name.replace('submission-', '').replace('submission_', '')[0:2]
    prefix_parts.append(tmp)
prefix_parts.sort()
prefix = 'avg_' + '-'.join(prefix_parts)
    
import datetime
now = datetime.datetime.now()
output_file = f'./output/ensemble/esb_{prefix}.csv'
sub['Survived'] =  ((sub['Survived'] / len(file_names)) >= 0.5).astype(int)
sub.to_csv(output_file, index=False)
print(output_file)
sub


./output/ensemble/esb_avg_21-33-44.csv


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [68]:
#  タイプ ２

# 加重平均で算出 weighted average
# 加重平均は、
# 値, 重みが、
# A, a
# B, b
# C, c
# とした場合、
# 加重平均 = (A*a + B*b + C*c) / (a+b+c)
# 

sub = pd.read_csv('input/gender_submission.csv')
sub['Survived'] = 0

file_names = [
    ('submission_48_tidycol_many_correlation_cate_plus_raw.csv',0.77272),
    ('submission_21-random-rorest.csv', 0.76794),
#     ('submission_33-lightgbm-stratified-kfold_raw.csv', 0.76555) ,
#     ('submission_44_tidycol_with_cabin_prefix_with_embarked_raw.csv', 0.76315),
    ('submission_50_tidycol_fillna_raw.csv', 0.76315),
#     ('submission_49_tidycol_many_correlation_cate_plus_b_raw.csv', 0.76315),
#     ('submission_45_tidycol_with_cabin_prefix_with_embarked_with_fare.csv', 0.76315),
# #     ('submission_46_tidycol_many_correlation_raw.csv', 0.76315),
#     ('submission_47_tidycol_many_correlation_with_sibsp_parch_raw.csv', 0.76076),
#     ('submission_32-lightgbm-change-hyper-param_raw.csv', 0.76076),
#     ('submission_42_tidycol_tidy_raw.csv', 0.76076),
#     ('submission_31-lightgbm-normal_raw.csv', 0.76076),
#     ('submission_32-lightgbm-change-hyper-param_raw.csv', 0.76076),
#     ('submission_41_tidycol_base_raw.csv', 0.75837),
#     ('submission_43_tidycol_with_cabin_prefix_raw.csv', 0.75119),
#     ('submission_11-with-familysize-logi-reg.csv', 0.65550),
]

file_dfs = []
weight_sum = 0
for (file_name, weight) in file_names:
    file_df = pd.read_csv(f'./output/submission/{file_name}')
    print(sub['Survived'])
    sub['Survived'] = sub['Survived'] + file_df['Survived'] * weight
    weight_sum += weight

print(sub['Survived'])
print(weight_sum)

# prefixを作る
prefix_parts = []
for (file_name, weight) in file_names:
    tmp = file_name.replace('submission-', '').replace('submission_', '')[0:2]
    prefix_parts.append(tmp)
prefix_parts.sort()
prefix = 'weighted-avg_' + '-'.join(prefix_parts)
    
import datetime
now = datetime.datetime.now()
output_file = f'./output/ensemble/esb_{prefix}.csv'
sub['Survived'] =  ((sub['Survived'] / weight_sum) >= 0.5).astype(int)
print(sub['Survived'])

sub.to_csv(output_file, index=False)
print(output_file)
sub


0      0
1      0
2      0
3      0
4      0
      ..
413    0
414    0
415    0
416    0
417    0
Name: Survived, Length: 418, dtype: int64
0      0.046702
1      0.489183
2      0.121582
3      0.100805
4      0.339170
         ...   
413    0.088975
414    0.749764
415    0.019910
416    0.088975
417    0.259623
Name: Survived, Length: 418, dtype: float64
0      0.046702
1      0.489183
2      0.121582
3      0.100805
4      1.107110
         ...   
413    0.088975
414    1.517704
415    0.019910
416    0.088975
417    0.259623
Name: Survived, Length: 418, dtype: float64
0      0.085789
1      0.946097
2      0.245073
3      0.148977
4      1.539092
         ...   
413    0.164363
414    2.218062
415    0.059019
416    0.164363
417    0.448208
Name: Survived, Length: 418, dtype: float64
2.30381
0      0
1      0
2      0
3      0
4      1
      ..
413    0
414    1
415    0
416    0
417    0
Name: Survived, Length: 418, dtype: int64
./output/ensemble/esb_weighted-avg_21-48-50.csv


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
